<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_ML_20210602/blob/main/%E9%A6%99%E8%95%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [290]:
import pandas as pd
# train_df = pd.read_csv("banana.csv", encoding="utf-8")
train_df = pd.read_excel("banana.xlsx")

# Date 民國 to 西元
def date_df(d):
  d = d.split("/")
  year = str(int(d[0])+1911)
  month = (d[1])
  day = (d[2])
  d = (f"{year}-{month}-{day}")
  return d
date_df = pd.DataFrame({"Date":train_df["Date"].apply(date_df)})
train_df.update(date_df)

# Month method
def Month(m):
  Month_data = int(m.split("-")[1])
  return Month_data
train_df["Month"] = train_df["Date"].apply(Month)

# Week_day method
def week_day(d):
  d = d.split("-")
  year = int(d[0])
  month = int(d[1])
  day = int(d[2])
  temp = pd.Timestamp(f"{year}-{month}-{day}")
  week_day = temp.dayofweek + 1 #temp.dayofweek 0(禮拜一) 開始, +1是因為比較直接禮拜幾
  return week_day
train_df["Week_day"] = train_df["Date"].apply(week_day)
train_df["Year"] = train_df["Date"].apply(lambda d:int(d.split("-")[0]))

# Rest_market
market_df = pd.DataFrame({"Market":train_df["Market"].apply(lambda t:t.split(" ")[1])})
train_df.update(market_df)

# Volume chnage from str to int
# train_df["Volume"] = train_df["Volume"].apply(lambda x:int(x.replace(",","")))

# Up_price change from onj to float
# train_df["Up_price"] = train_df["Up_price"].apply(lambda x:float(x))
market_df["Market"]
train_df

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year
0,2000-01-02,台北二,A1 香蕉,32.9,21.3,20.0,23.3,6531.0,1,7,2000
1,2000-01-03,台北二,A1 香蕉,32.2,21.8,17.7,23.0,4971.0,1,1,2000
2,2000-01-04,台北二,A1 香蕉,31.6,22.5,13.7,22.6,5192.0,1,2,2000
3,2000-01-05,台北二,A1 香蕉,31.0,22.0,18.1,23.0,4102.0,1,3,2000
4,2000-01-06,台北二,A1 香蕉,30.0,24.4,18.3,24.3,6243.0,1,4,2000
...,...,...,...,...,...,...,...,...,...,...,...
58979,2021-06-12,台東市,A1 香蕉,32.0,21.5,15.0,22.8,3007.0,6,6,2021
58980,2021-06-13,台東市,A1 香蕉,30.0,26.2,18.0,27.9,3558.0,6,7,2021
58981,2021-06-14,台東市,A1 香蕉,30.0,26.2,18.0,22.9,2857.0,6,1,2021
58982,2021-06-17,台東市,A1 香蕉,33.0,22.9,15.0,20.8,3052.0,6,4,2021


In [291]:
train_df["Date"] = pd.to_datetime(train_df["Date"])
train_df["Date"]

0       2000-01-02
1       2000-01-03
2       2000-01-04
3       2000-01-05
4       2000-01-06
           ...    
58979   2021-06-12
58980   2021-06-13
58981   2021-06-14
58982   2021-06-17
58983   2021-06-19
Name: Date, Length: 58984, dtype: datetime64[ns]

In [292]:
train_df
train_df.dtypes

Date         datetime64[ns]
Market               object
Product              object
Up_price            float64
Mid_price           float64
Low_price           float64
Avg_price           float64
Volume              float64
Month                 int64
Week_day              int64
Year                  int64
dtype: object

In [342]:
market_list = train_df["Market"].value_counts().index.tolist()
m_total = []
for i,mark_name in enumerate(market_list):
  a = train_df["Market"] == mark_name
  start_time = train_df[a].reset_index(drop=True)["Date"][0]
  t_index = pd.date_range(start_time, '2021-06-19', freq='D') #start_time by 市場別
  T_df = pd.DataFrame(t_index, columns=['Date'])
  m = train_df[a].merge(T_df, how="right",sort=True)
  most = m.value_counts().idxmax()
  m["Market"]=m["Market"].fillna(most[1])
  m["Product"]=m["Product"].fillna(most[2])
  na = m["Volume"].isna()
  m["Month"] = m["Date"].apply(lambda x:int(str(x).split("-")[1]))
  m["Week_day"] = m["Date"].apply(lambda x:pd.Timestamp(str(x).split()[0]).dayofweek+1)
  m["Year"] = m["Date"].apply(lambda x:int(str(x).split("-")[0]))
  import math
  def restday(s):
    if s > 0:
      return 0
    else:
      return 1
  m["Rest_day"] = m["Volume"].apply(restday) #判斷 restday 0 or 1
  globals()[f"m{i}"] = m
  m_total.append(globals()[f"m{i}"])
print(m.dtypes)
len(market_list)
final_data = pd.concat(m_total).reset_index(drop=True)
# final_data.to_csv("banana_final.csv")
final_data.to_excel("banana_final.xlsx")

Date         datetime64[ns]
Market               object
Product              object
Up_price            float64
Mid_price           float64
Low_price           float64
Avg_price           float64
Volume              float64
Month                 int64
Week_day              int64
Year                  int64
Rest_day              int64
dtype: object


In [343]:
final_data

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year,Rest_day
0,2000-01-01,嘉義市,A1 香蕉,17.0,12.0,10.0,12.0,650.0,1,6,2000,0
1,2000-01-02,嘉義市,A1 香蕉,17.0,12.0,10.0,12.0,760.0,1,7,2000,0
2,2000-01-03,嘉義市,A1 香蕉,17.0,12.0,10.0,12.0,810.0,1,1,2000,0
3,2000-01-04,嘉義市,A1 香蕉,17.0,12.0,10.0,12.0,970.0,1,2,2000,0
4,2000-01-05,嘉義市,A1 香蕉,18.0,12.0,10.0,12.0,1080.0,1,3,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
80519,2021-06-15,東勢區,A1 香蕉,NaN,NaN,NaN,NaN,NaN,6,2,2021,1
80520,2021-06-16,東勢區,A1 香蕉,NaN,NaN,NaN,NaN,NaN,6,3,2021,1
80521,2021-06-17,東勢區,A1 香蕉,NaN,NaN,NaN,NaN,NaN,6,4,2021,1
80522,2021-06-18,東勢區,A1 香蕉,13.3,13.3,13.3,13.3,18.6,6,5,2021,0
